In [2]:
!pip install datasets evaluate transformers seqeval peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 15.6 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=943f8216c0f33cc619d7a0502305b9a3f2551b669b4ab37eee732e6dc3d8f050
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [4]:
model_name = "xlm-roberta-large"
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from transformers import DataCollatorForTokenClassification
from datasets import Dataset
from peft import get_peft_model, LoraConfig
import evaluate

model_name = model_name  


# Load dataset and process data
def load_conll_data(file_path, is_test=False):
    sentences = []
    tokens, tags = [], []

    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith("#") or line.strip() == "":
                if tokens:
                    sentences.append((tokens, tags if not is_test else []))  # Append empty tags if test
                    tokens, tags = [], []
                continue
            parts = line.strip().split('\t')
            if len(parts) == 3:
                token, _, tag = parts
                tokens.append(token)
                if not is_test:
                    tags.append(tag) 
    if tokens:
        sentences.append((tokens, tags if not is_test else []))

    return sentences



train_sentences = load_conll_data('/kaggle/input/ner-hineng/train.conll')
dev_sentences = load_conll_data('/kaggle/input/ner-hineng/dev.conll')
test_sentences = load_conll_data('/kaggle/input/ner-hineng/test.conll', is_test=True)

print(train_sentences[0])

# Create a mapping of tags to IDs
def create_label_mapping(sentences):
    tags = set()
    for _, sentence_tags in sentences:
        if sentence_tags:
            tags.update(sentence_tags)
    label2id = {label: i for i, label in enumerate(sorted(tags))}
    return label2id


label2id = create_label_mapping(train_sentences)

print(label2id)

(['Gully', 'cricket', 'chal', 'raha', 'hain', 'yaha', '"', '(', 'Soniya', ')', 'Gandhi', '"'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PERSON', 'O', 'B-PERSON', 'O'])
{'B-ORGANISATION': 0, 'B-PERSON': 1, 'B-PLACE': 2, 'I-ORGANISATION': 3, 'I-PERSON': 4, 'I-PLACE': 5, 'O': 6}


In [5]:
def convert_to_hf_dataset(sentences):
    data = {'id': [], 'tokens': [], 'ner_tags': []}
    for idx, (tokens, tags) in enumerate(sentences):
        data['id'].append(idx)  # Use a simple integer ID
        data['tokens'].append(tokens)
        data['ner_tags'].append(tags)
    return Dataset.from_dict(data)


train_dataset = convert_to_hf_dataset(train_sentences)
dev_dataset = convert_to_hf_dataset(dev_sentences)
test_dataset = convert_to_hf_dataset(test_sentences)

print(train_dataset[0])

unique_tags = set(tag for _, tags in train_sentences for tag in tags)

tag2id = {
    'O': 0,
    'B-PERSON': 1,
    'I-PERSON': 2,
    'B-ORGANISATION': 3,
    'I-ORGANISATION': 4,
    'B-PLACE': 5,
    'I-PLACE': 6,
}

id2tag = {idx: tag for tag, idx in tag2id.items()}

def convert_labels_to_ids(example):
    example['ner_tags'] = [tag2id.get(label, -1) for label in example['ner_tags']]  
    return example


train_dataset = train_dataset.map(convert_labels_to_ids)
dev_dataset = dev_dataset.map(convert_labels_to_ids)
test_dataset = test_dataset.map(convert_labels_to_ids)
print(len(test_dataset))
print("Tag to ID mapping:")
for tag, id in tag2id.items():
    print(f"Tag: {tag}, ID: {id}")

print("\nID to Tag mapping:")
for id, tag in id2tag.items():
    print(f"ID: {id}, Tag: {tag}")

example = train_dataset[0]
print(example)

label_list = list(tag2id.keys())
print(label_list)

print(train_dataset.column_names)

{'id': 0, 'tokens': ['Gully', 'cricket', 'chal', 'raha', 'hain', 'yaha', '"', '(', 'Soniya', ')', 'Gandhi', '"'], 'ner_tags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PERSON', 'O', 'B-PERSON', 'O']}


Map:   0%|          | 0/1678 [00:00<?, ? examples/s]

Map:   0%|          | 0/433 [00:00<?, ? examples/s]

0
Tag to ID mapping:
Tag: O, ID: 0
Tag: B-PERSON, ID: 1
Tag: I-PERSON, ID: 2
Tag: B-ORGANISATION, ID: 3
Tag: I-ORGANISATION, ID: 4
Tag: B-PLACE, ID: 5
Tag: I-PLACE, ID: 6

ID to Tag mapping:
ID: 0, Tag: O
ID: 1, Tag: B-PERSON
ID: 2, Tag: I-PERSON
ID: 3, Tag: B-ORGANISATION
ID: 4, Tag: I-ORGANISATION
ID: 5, Tag: B-PLACE
ID: 6, Tag: I-PLACE
{'id': 0, 'tokens': ['Gully', 'cricket', 'chal', 'raha', 'hain', 'yaha', '"', '(', 'Soniya', ')', 'Gandhi', '"'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]}
['O', 'B-PERSON', 'I-PERSON', 'B-ORGANISATION', 'I-ORGANISATION', 'B-PLACE', 'I-PLACE']
['id', 'tokens', 'ner_tags']


In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
example = train_dataset[0]
tokenized_inputs = tokenizer(
    example["tokens"],
    is_split_into_words=True,
    padding=True,
    truncation=True,
    return_tensors='pt',
)
tokens = tokenizer.convert_ids_to_tokens(tokenized_inputs['input_ids'][0].tolist())
print(tokens)
print(unique_tags)


def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


tokenized_train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_dev_dataset = dev_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)

tokenized_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_dev_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
print(train_dataset.column_names)  
print(tokenized_train_dataset.column_names)

print(train_dataset[0])
print(tokenized_train_dataset[1])

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True)

metric = evaluate.load("seqeval")
import numpy as np

labels = [label_list[i] for i in example["ner_tags"]]
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score

from seqeval.scheme import BILOU


from seqeval.metrics import f1_score


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

['<s>', '▁Gul', 'ly', '▁cri', 'cket', '▁chal', '▁raha', '▁hain', '▁ya', 'ha', '▁"', '▁(', '▁Son', 'iya', '▁)', '▁Gandhi', '▁"', '</s>']
{'O', 'I-PLACE', 'B-PLACE', 'I-ORGANISATION', 'B-PERSON', 'I-PERSON', 'B-ORGANISATION'}


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/1678 [00:00<?, ? examples/s]

Map:   0%|          | 0/433 [00:00<?, ? examples/s]

['id', 'tokens', 'ner_tags']
['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels']
{'id': 0, 'tokens': ['Gully', 'cricket', 'chal', 'raha', 'hain', 'yaha', '"', '(', 'Soniya', ')', 'Gandhi', '"'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]}
{'input_ids': tensor([     0,     87,   1884,     71,     10,   1374, 140668,   1202,   3975,
           696,     18,      5,    587,     64,    397,     39,    856,    864,
           127,   1511,    275,     85,    568, 195902,  12530,   1716,   1460,
            62,     11, 132507,   4368,      9, 117117,  18385,     20,  36635,
          6827,  76417,   3311,  56174,  16785,    936,      2]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 'labels': tensor([-100,    0,    0, -100,    0,    0, -100,    0,    0, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,    0,
  

In [7]:
lora_config = LoraConfig(
    r=8,  
    task_type="TOKEN_CLASSIFICATION",
    lora_alpha=32, 
    target_modules=["query", "value"],  
    lora_dropout=0.1,  
)

model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(tag2id), id2label=id2tag,label2id=tag2id)
model = get_peft_model(model, lora_config)

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
import evaluate
def compute_metrics(p):
    print("in compute metrics")
    preds = [[id2tag[i] for i in pred] for pred in p.predictions]
    refs = [[id2tag[i] for i in ref] for ref in p.labels]

    # Remove ignored index (usually -100)
    true_predictions = [
        [pred for pred, label in zip(prediction, label) if label != -100]
        for prediction, label in zip(preds, refs)
    ]
    true_labels = [
        [label for pred, label in zip(prediction, label) if label != -100]
        for prediction, label in zip(preds, refs)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }



In [9]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

print(len(tokenized_dev_dataset))

print(tokenized_dev_dataset[0])

from transformers import TrainingArguments, Trainer
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

433
{'input_ids': tensor([     0, 110833,  25813,      6,      4, 179635,   2363,      6,  66398,
         43904,    136,  92635,    111,     10,  23009,  18940,  37772,      6,
             5,   3975,    696,     18,      5,    587,     64,  46592,    910,
           434,    238,    866,    299,     92,    397,      2]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 'labels': tensor([-100,    0,    0,    0, -100,    0,    0,    0, -100,    0,    0,    0,
           0,    0,    0,    0,    0,    0, -100,    0, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100])}


In [10]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    num_train_epochs=1,
    do_train=True,
    report_to=[],  
    remove_unused_columns=False, 
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate = 2e-5,
    weight_decay = 0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_dev_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

from transformers import TrainerCallback


class PrintMetricsCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics:
            # Print F1 and other evaluation metrics
            print(f"Epoch {int(state.epoch)} - Evaluation Metrics:")
            print(f"  F1 Score: {metrics.get('eval_f1', 'N/A')}")
            print(f"  Precision: {metrics.get('eval_precision', 'N/A')}")
            print(f"  Recall: {metrics.get('eval_recall', 'N/A')}")
            print(f"  Accuracy: {metrics.get('eval_accuracy', 'N/A')}\n")



print_metrics_callback = PrintMetricsCallback()


print(tokenized_dev_dataset[0])

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


{'input_ids': tensor([     0, 110833,  25813,      6,      4, 179635,   2363,      6,  66398,
         43904,    136,  92635,    111,     10,  23009,  18940,  37772,      6,
             5,   3975,    696,     18,      5,    587,     64,  46592,    910,
           434,    238,    866,    299,     92,    397,      2]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 'labels': tensor([-100,    0,    0,    0, -100,    0,    0,    0, -100,    0,    0,    0,
           0,    0,    0,    0,    0,    0, -100,    0, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100])}


In [11]:
def evaluate_ner(trainer, eval_dataset):
    predictions_tuple = trainer.predict(eval_dataset)
    logits = predictions_tuple.predictions[1]

    if len(logits.shape) == 3:
        predicted_labels = np.argmax(logits, axis=-1)
    else:
        raise ValueError(f"Unexpected shape for logits: {logits.shape}")

    true_labels_flat = []
    predicted_labels_flat = []
    padding_label = -100  # Assuming -100 is used for ignored labels

    for i in range(len(eval_dataset)):
        true_label_seq = eval_dataset[i]["labels"]
        pred_label_seq = predicted_labels[i]

        # Ignore padding labels and -100 labels in true labels
        true_labels_flat.extend(
            [label for label in true_label_seq if label != padding_label and label != 0])  # Adjusting for 0 as well
        predicted_labels_flat.extend(
            [pred for pred, label in zip(pred_label_seq, true_label_seq) if label != padding_label and label != 0])

    true_labels_flat = np.array(true_labels_flat)
    predicted_labels_flat = np.array(predicted_labels_flat)

    if len(predicted_labels_flat) != len(true_labels_flat):
        print(
            f"Warning: Predicted labels length ({len(predicted_labels_flat)}) does not match true labels length ({len(true_labels_flat)}). Adjusting...")
        min_length = min(len(predicted_labels_flat), len(true_labels_flat))
        predicted_labels_flat = predicted_labels_flat[:min_length]
        true_labels_flat = true_labels_flat[:min_length]

    precision = precision_score(true_labels_flat, predicted_labels_flat, average="macro", zero_division=0)
    recall = recall_score(true_labels_flat, predicted_labels_flat, average="macro", zero_division=0)
    f1 = f1_score(true_labels_flat, predicted_labels_flat, average="macro", zero_division=0)
    accuracy = accuracy_score(true_labels_flat, predicted_labels_flat)

    print(
        f"Evaluation Metrics:\n F1: {f1:.4f}\n Precision: {precision:.4f}\n Recall: {recall:.4f}\n Accuracy: {accuracy:.4f}")
    return {"f1": f1, "precision": precision, "recall": recall, "accuracy": accuracy}

In [12]:
import os

best_model_dir = "./best_model"
os.makedirs(best_model_dir, exist_ok=True)

In [13]:
maxf1 = 0
savemodel = model
maxidx = 0

In [16]:
for i in range(50):
  print(101 + i)
  trainer.train()
  eval_results = evaluate_ner(trainer, tokenized_dev_dataset)
  if(eval_results['f1'] > maxf1):
      maxf1 = eval_results['f1']
      savemodel = model
      maxidx = i
      model.save_pretrained(best_model_dir)
      tokenizer.save_pretrained(best_model_dir)
  print("Final Evaluation Results:", eval_results)

101


Epoch,Training Loss,Validation Loss
1,0.062600,No log


Evaluation Metrics:
 F1: 0.6750
 Precision: 0.7410
 Recall: 0.6430
 Accuracy: 0.8226
Final Evaluation Results: {'f1': 0.6749926877869435, 'precision': 0.7410338191371479, 'recall': 0.6429587783562061, 'accuracy': 0.8226086956521739}
102


Epoch,Training Loss,Validation Loss
1,0.064900,No log


Evaluation Metrics:
 F1: 0.6749
 Precision: 0.7331
 Recall: 0.6409
 Accuracy: 0.8330
Final Evaluation Results: {'f1': 0.6749407711265885, 'precision': 0.7331138798991547, 'recall': 0.6408583712642353, 'accuracy': 0.8330434782608696}
103


...

Evaluation Metrics:
 F1: 0.7048
 Precision: 0.7520
 Recall: 0.6698
 Accuracy: 0.8365
Final Evaluation Results: {'f1': 0.7047661400652595, 'precision': 0.7519652767867052, 'recall': 0.6698385128442822, 'accuracy': 0.8365217391304348}
149


Epoch,Training Loss,Validation Loss
1,0.050000,No log


Evaluation Metrics:
 F1: 0.6920
 Precision: 0.7534
 Recall: 0.6488
 Accuracy: 0.8174
Final Evaluation Results: {'f1': 0.6919622033663279, 'precision': 0.7534352033450807, 'recall': 0.6487793416538451, 'accuracy': 0.8173913043478261}
150


Epoch,Training Loss,Validation Loss
1,0.049500,No log


Evaluation Metrics:
 F1: 0.6938
 Precision: 0.7518
 Recall: 0.6530
 Accuracy: 0.8226
Final Evaluation Results: {'f1': 0.6938201528835056, 'precision': 0.7518321474226662, 'recall': 0.6530442359425274, 'accuracy': 0.8226086956521739}


In [17]:
print(maxf1)
print(maxidx)

0.7205274824448245
10
